In [1]:
import pandas as pd


In [2]:
new = pd.read_csv("Walmart.csv")

In [3]:
new

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106
...,...,...,...,...,...,...,...,...
6430,45,28-09-2012,713173.95,0,64.88,3.997,192.013558,8.684
6431,45,05-10-2012,733455.07,0,64.89,3.985,192.170412,8.667
6432,45,12-10-2012,734464.36,0,54.47,4.000,192.327265,8.667
6433,45,19-10-2012,718125.53,0,56.47,3.969,192.330854,8.667


In [4]:
new.isnull().sum()

Store           0
Date            0
Weekly_Sales    0
Holiday_Flag    0
Temperature     0
Fuel_Price      0
CPI             0
Unemployment    0
dtype: int64

In [4]:
import h2o
from h2o.automl import H2OAutoML

In [5]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 hour 24 mins
H2O_cluster_timezone:,Europe/London
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.1
H2O_cluster_version_age:,1 month
H2O_cluster_name:,H2O_from_python_lakshmiph_822sg8
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.541 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [6]:
# convert pandas DataFrame into H2O Frame
new_df = h2o.H2OFrame(new)
# Describe  the train h20Frame
new_df.describe()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Rows:6435
Cols:8

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
type,int,enum,real,int,real,real,real,real
mins,1.0,,209986.25,0.0,-2.06,2.472,126.064,3.879
mean,23.000000000000068,,1046964.8775617725,0.06993006993006994,60.663782439782516,3.3586068376068403,171.57839384878065,7.999151048951066
maxs,45.0,,3818686.45,1.0,100.14,4.468,227.2328068,14.313
sigma,12.988182381175454,,564366.6220536977,0.25504894436982795,18.444932875811585,0.45901970719285223,39.356712295664195,1.8758847818627944
zeros,0,,0,5985,0,0,0,0
missing,0,0,0,0,0,0,0,0
0,1.0,05-02-2010,1643690.9,0.0,42.31,2.572,211.0963582,8.106
1,1.0,12-02-2010,1641957.44,1.0,38.51,2.548,211.2421698,8.106
2,1.0,19-02-2010,1611968.17,0.0,39.93,2.514,211.2891429,8.106


In [7]:
y = "Unemployment"
x = new_df.columns
# remove label classvariable from feature variable
x.remove(y)

In [8]:
splits = new_df.split_frame(ratios = [0.8], seed = 1)
train = splits[0]
test = splits[1]

In [9]:
aml = H2OAutoML(max_models=50, seed = 1, project_name = "walmart")
aml.train(x = x, y = y, training_frame = train, validation_frame=test)

AutoML progress: |
20:05:22.364: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
20:05:22.365: AutoML: XGBoost is not available; skipping it.
20:10:02.990: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
20:10:02.990: New models will be added to existing leaderboard walmart@@Unemployment (leaderboard frame=null) with already 101 models.
20:10:02.990: AutoML: XGBoost is not available; skipping it.
20:10:04.904: StackedEnsemble_BestOfFamily_6_AutoML_9_20230720_201002 [StackedEnsemble best_of_family_1 (built with AUTO metalearner, using top model from each algorithm type)] fai

██████████████████████████████████████████████████████████████ (cancelled) 100%


KeyboardInterrupt: 